In [129]:
import numpy as np
import networkx as nx
from collections import defaultdict
import scipy.io
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from scipy.sparse.linalg import svds
from copy import deepcopy
from collections import Counter
from tqdm import tqdm
import random

In [130]:
import os
os.chdir('/content/drive/My Drive/wstm/code')
!pwd

/content/drive/My Drive/wstm/code


In [131]:
p=0.25 # Return parameter
q=0.25 # In-out parameter
walks=20 # Walks per node
length=30 # Length of each walk
d=10 # Dimension of output embeddings
window=64 # Window size for word2vec
workers=4 # Number of workers to assign for random walk and word2vec


In [132]:
def compute_transition_probs(G, probs, p, q):
    """
    Inputs: 
    G: nx.graph
    probs: this is a dictionary of transition scores that you will fill. 
    p: the return parameter
    q: the in-out parameter
    
    Follow the formula above to create transition_probs. 

    Steps to follow: 
    1. Iterate over all nodes s in G.
    2. In each iteration, iterate over all neighbors c of node s.
    3. For every neighbor d of node c, calculate the probability of a random walker
       visiting node d from node c if the random walker was on node s in the previous step. 
    4. Store the probability values in transition_probs. 

    Return: 
    transition_probs: This is a dictionary of transition probabilities during the random walk. 
    This is a nested dictionary.  
    transition_probs[s][c] = a list of normalized probability scores of visiting node d from node s via node c. 
    
    The format of transition_probs is given after this cell. 
    """
    transition_probs = deepcopy(probs)
    for s in list(G.nodes()):
        nbrs_s = list(G.neighbors(s))
        for c in nbrs_s:
            probs_list = []
            nbrs_c = list(G.neighbors(c))
            for d in nbrs_c:
                if d==s:
                    probs_list.append((d,1/p))
                elif d in nbrs_s:
                    probs_list.append((d,1))
                else:
                    probs_list.append((d,1/q))
            probs = np.array([item[1] for item in probs_list])
            transition_probs[str(s)][str(c)] = [(item[0],item[1]/np.sum(probs)) for item in probs_list]
    return transition_probs

In [156]:
def generate_walks(G, transition_probs, walks_per_node, length):

    walks = []
    for s in tqdm(G.nodes(data=True)):
        for walk_number in range(0,walks_per_node):
            #if user select neighbour randomly 
            if str.startswith(s[0],'user'):
                nbrs_s = list(G.neighbors(s[0]))
                #movie will be chosen randomly
                c = np.random.choice(nbrs_s)
                this_walk = [s[0],c]
                while(len(this_walk)<length):
                    this_s = this_walk[-2]
                    this_c = this_walk[-1]
                    
                    nbrs_c = list(G.neighbors(this_c))
                    if len(nbrs_c) == 0:
                        break
                    if str.startswith(this_c,'movie'):
                    #if c is a movie neighbours of c will be users
                        # print(this_c,":",nbrs_c) 
                        men = [x for x in nbrs_c if G.nodes[x]['gender']=='M']
                        women  = [x for x in nbrs_c if G.nodes[x]['gender']=='F']
                        toss = random.random()
                        if toss >= 0.5:
                            if len(women) > 0:
                                nbrs_c = women
                            else:
                                nbrs_c = men
                        else:
                            if len(men) > 0:
                                nbrs_c = men
                            else:
                                nbrs_c = women   
                    #if c is a user neighbours of c will be movies
                    d = np.random.choice(nbrs_c)
                    this_walk.append(d)

            else:
                #neighbours of s are users 
                nbrs_s = list(G.neighbors(s[0]))
                if len(nbrs_s) == 0:
                    break
                men = [x for x in nbrs_s if G.nodes[x]['gender']=='M']
                women  = [x for x in nbrs_s if G.nodes[x]['gender']=='F']
                toss = random.random()
                if toss >= 0.5:
                    if len(women) > 0:
                        nbrs_s = women
                    else:
                        nbrs_s = men
                else:
                    if len(men) > 0:
                        nbrs_s = men
                    else:
                        nbrs_s = women
                c = np.random.choice(nbrs_s)
                this_walk = [s[0],c]

                while(len(this_walk)<length):
                    this_s = this_walk[-2]
                    this_c = this_walk[-1]
                    
                    nbrs_c = list(G.neighbors(this_c))
                    if len(nbrs_c) == 0:
                        break
                    if str.startswith(this_c,'movie'):
                    #if c is a movie neighbours of c will be users 
                        men = [x for x in nbrs_c if G.nodes[x]['gender']=='M']
                        women  = [x for x in nbrs_c if G.nodes[x]['gender']=='F']
                        toss = random.random()
                        if toss >= 0.5:
                            if len(women) > 0:
                                nbrs_c = women
                            else:
                                nbrs_c = men
                        else:
                            if len(men) > 0:
                                nbrs_c = men
                            else:
                                nbrs_c = women   
                    #if c is a user neighbours of c will be movies
                    d = np.random.choice(nbrs_c)
                    this_walk.append(d)
            walks.append(this_walk)
    walks = [[str(j) for j in walk] for walk in walks]
    np.random.shuffle(walks)
    return walks

In [157]:
def generate_embeddings(walks, dimensions, window_size, num_workers, p, q):
    """
    Here we use word2vec code to generate node embeddings from the random walks. 
    Please refer to https://radimrehurek.com/gensim/models/word2vec.html for more information about word2vec.
    
    walks: Simulated random walks
    dimensions: Output dimension of node embeddings
    window_size: Window size for word2vec
    num_workers: Number of workers to assign for random walk and word2vec
    p: Return parameter
    q: In out parameter
    
    Return:
    model: the learned word2vec model
    embeddings: embeddings of all nodes generated using the word2vec model
    """
    model=None
    embeddings=None
    model = Word2Vec(walks, size=dimensions, window=window_size, min_count=0, sg=1, workers=num_workers,seed=0)
    embeddings = model.wv

    return model, embeddings

In [158]:
def fairwalk(G, probs, p, q, walks, length, workers, d, window, trans_probs):
  """
  Generate embeddings for node2vec using the above functions you just implemented. 
  Inputs:
  input: Path for input
  directed: True if you want a directed graph, False if you want a undirected graph
  p: Return parameter
  q: In out parameter
  walks: Length of walks for each node in graph G
  length: Length of each walk
  workers: Number of workers to assign for random walk and word2vec
  d: Output dimension of node embeddings
  window: Window size for word2vec

  Steps to follow:
  1. Call compute_transition_probs.
  2. Call generate_walks.
  3. Call generate_embeddings. 

  Return:
  walk: simulated walks from each node
  model: word2vec model
  node2vec_embeddings: embeddings generated from the model
  """

  model=None
  node2vec_embeddings=None
  if trans_probs:
    print("\nBegin computing transition probabilities")
    transition_probs = compute_transition_probs(G, probs, p, q)
    print("\nComputed transition probabilities")
  else:
    transition_probs = None    
  walk = generate_walks(G, transition_probs, walks, length)
  print("\nGenerated walks")
  model, node2vec_embeddings = generate_embeddings(walk, d, window, workers, p, q) 
  #######################################
  return walk, model, node2vec_embeddings

In [159]:
G = nx.read_gpickle("../data/ml_graph.gpickle")
probs = defaultdict(dict)
for node in G.nodes():
    probs[str(node)] = dict()
walk, model, fairwalk_embeddings = fairwalk(G, probs, p, q, walks, length, workers, d, window, False)

100%|██████████| 2625/2625 [05:03<00:00,  8.66it/s]



Generated walks


In [160]:
# Look for most similar nodes
print(model.wv.most_similar('user1')) # Output node names are always strings

# Save embeddings for later use
model.wv.save_word2vec_format('../data/ml_fairwalk_emebddings.model')

# Save model for later use
model.save('../data/ml_fairwalk_model.embeddings')

[('user622', 0.9815174341201782), ('user286', 0.9725745916366577), ('user847', 0.9654041528701782), ('user542', 0.9651858806610107), ('user805', 0.9638780355453491), ('user94', 0.9603023529052734), ('user758', 0.9596209526062012), ('user778', 0.9592470526695251), ('user795', 0.9592008590698242), ('user11', 0.9559310674667358)]
